# TIØ4317 Project

Group members:
- Frederick Johan Pretlove
- Fabian Stoltenberg
- Hans Kristian Bjørgo Kværum
- Hedda Flemmen Holum

## Problem statement

This project investigates how fluctuations in the oil price, and the Norwegian Krone (NOK) exchange rate influence the stock prices of selected companies listed on the Oslo Stock Exchange (OSE). By applying time-series analysis, we aim to determine the explanatory power of these external economic factors in stock price prediction.

We will retrieve historical data via APIs (Yahoo Finance and Norges Bank) and implement ARIMA and ARIMAX models (see explanations in the “Empirical Analysis” section). Our goal is to compare their predictive performance and assess whether incorporating oil prices and exchange rates improves stock price forecasts beyond a baseline ARIMA model. If these external factors enhance predictive accuracy, this would indicate a relationship between commodity prices, exchange rates, and stock prices.

To ensure model robustness, we will conduct diagnostic tests for stationarity, autocorrelation, and heteroscedasticity. The findings will provide insights into the relationship between macroeconomic factors and energy stock prices, offering valuable guidance to investors and analysts seeking to better understand market dynamics.
